In [1]:
import os
import torch

from torchinfo import summary

In [2]:
from models import make_resnet_pretrained_classifier

: 

: 

In [3]:
from lared_laughter.constants import models_path

In [4]:
chckpt = torch.load(os.path.join(models_path, 'SLOW_4x16_R50.pyth'))

In [2]:
pretrained = make_resnet_pretrained_classifier()

RuntimeError: Error(s) in loading state_dict for Net:
	size mismatch for blocks.0.conv.weight: copying a param with shape torch.Size([64, 3, 1, 7, 7]) from checkpoint, the shape in current model is torch.Size([64, 3, 3, 7, 7]).

In [15]:
input = [
    torch.rand((8, 3, 8, 256, 256)), # slow
    torch.rand((8, 3, 32, 256, 256)) # fast
]

In [16]:
summary(pretrained, input_data=[input], device='cpu')

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [ResNetBasicStem: 4, Conv3d: 5, BatchNorm3d: 5, ReLU: 5, MaxPool3d: 5, ResNetBasicStem: 4, Conv3d: 5, BatchNorm3d: 5, ReLU: 5, MaxPool3d: 5, Conv3d: 4, BatchNorm3d: 4, ReLU: 4]